  # VA Data tools
  ## Tools for navigating VA Datasets
  
  ## History
The VA open data portal (https://www.data.va.gov/) contains several thousand data files in Excel, CSV, and PDF formats.  Brent Brewington, using R, flattened out the dataset inventory from JSON to a CSV file.  Both the CSV file and the R code used to do this can be found at his repository:
https://github.com/bbrewington/VA-open-data-mysandbox



In [7]:
import csv, re, requests, io
import pandas as pd

We will download the latest version of Brent's CSV file and save it 'locally.' I only need to do this if something changes, really.

In [8]:
VADataInventory = requests.get('https://raw.githubusercontent.com/bbrewington/VA-open-data-mysandbox/master/va_data_inventory_links_checked_20171202.csv')
VADataInventory.raise_for_status()

megadata = open('va_data_inventory.csv', 'wb')
for chunk in VADataInventory.iter_content(100000):
    megadata.write(chunk)
    
megadata.close

<function BufferedWriter.close>

Did it work?

In [9]:
!dir

Initial_Exploration.ipynb  plot_ptsd_by_state.py   templates
README.md		   plot_va_locations.py    va_data_inventory.csv
VA-Data_Tools.ipynb	   populate_database.py    va_locations.html
VADataByState.csv	   ptsd_by_state.html
app.py			   ptsd_rate_by_state.sql


Cool!  When the file is downloaded, I'll just run from the following lines.  The use for these will become more clear as we dig through the various functions.

In [3]:
MegaData= open('va_data_inventory.csv', encoding = "utf8")
megasheet = csv.reader(MegaData) # read this using the CSV module
alldata = list(megasheet) # which is easier to work with in a list format
headers = alldata[0]      # easily grab the headers this way
file_extension_regex = re.compile(r'\.((pdf|csv|xlsx?|zip|asp))', re.IGNORECASE) 
states = ['Alabama' ,'Alaska' ,'Arizona' ,'Arkansas' ,'California' ,'Colorado' ,'Connecticut' ,'Delaware' ,'Florida' ,'Georgia' ,'Hawaii' ,'Idaho' ,'Illinois' ,'Indiana' ,'Iowa' ,'Kansas' ,'Kentucky' ,'Louisiana' ,'Maine' ,'Maryland' ,'Massachusetts' ,'Michigan' ,'Minnesota' ,'Mississippi' ,'Missouri' ,'Montana' ,'Nebraska' ,'Nevada' ,'New Hampshire' ,'New Jersey' ,'New Mexico' ,'New York' ,'North Carolina' ,'North Dakota' ,'Ohio' ,'Oklahoma' ,'Oregon' ,'Pennsylvania' ,'Rhode' 'Island' ,'South Carolina' ,'South Dakota' ,'Tennessee' ,'Texas' ,'Utah' ,'Vermont' ,'Virginia' ,'Washington' ,'West' 'Virginia' ,'Wisconsin' ,'Wyoming']
territories = ['American Samoa', 'Guam', 'Northern Mariana Islands', 'Puerto Rico']
states_territories = states + territories
StateDict= {"AL":"Alabama", "AS" : "American Samoa", "AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut", "DC": "District of Columbia", "DE":"Delaware","FL":"Florida","FM": "Federated States of Micronesia", "GA":"Georgia","HI":"Hawaii","GU": "Guam", "ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MH": "Marshall Islands", "MI":"Michigan","MN":"Minnesota","MP" : "Northern Mariana Islands", "MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","PR" : "Puerto Rico", "RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","VI": "Version Islands", "WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}



Great! First, note that Brent's CSV file contains the following headers:

In [22]:
headers

['@type',
 'accessLevel',
 'bureauCode',
 'dataQuality',
 'description',
 'identifier',
 'issued',
 'keyword',
 'license',
 'modified',
 'programCode',
 'theme',
 'title',
 'landingPage',
 'accrualPeriodicity',
 'language',
 'temporal',
 'describedBy',
 'references',
 'isPartOf',
 'describedByType',
 'systemOfRecords',
 'rights',
 'conformsTo',
 'contactPoint_fn',
 'contactPoint_hasEmail',
 'publisher_@type',
 'publisher_name',
 'distribution_@type',
 'distribution_downloadURL',
 'distribution_mediaType',
 'distribution_format',
 'distribution_description',
 'distribution_title',
 'distribution_accessURL',
 'distribution_accessURL_format',
 'link_status_20171202',
 'url']

Without losing too much by 'overcleaning' the CSV file, these tools will help us browse through it. Remember that we are now treating this CSV file in list format. So each row is a list, and each data element is an element in the list matching the headers.  i.e

In [23]:
alldata[1]

['dcat:Dataset',
 'public',
 '029:00',
 'TRUE',
 'This dataset provide a count of Veteran by their religious affiliation and state of residence. The dataset set covers all 50 states, District of Columbia and other territories.',
 '4b94ff43-3b72-4cd5-9894-ccfe8fc58716',
 '2017-08-16',
 'Religion; veterans',
 'https://creativecommons.org/publicdomain/zero/1.0/',
 '2017-08-16',
 '029:086',
 'Socioeconomics',
 'Veteran Religious Affiliation by State',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'NCVAS Mailbox',
 'mailto:vancvas@va.gov',
 'org:Organization',
 'Department of Veterans Affairs',
 'dcat:Distribution',
 'https://www.data.va.gov/sites/default/files/NCVAS%20Religion%20by%20State.xlsx',
 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
 'xlsx',
 '<p>This dataset provide a count of Veteran by their religious affiliation and state of residence. The dataset set covers all 50 states, District of Columbia and other territories.</p>\n',
 'NCVAS Religion By

The tools will help us sort through all this without 'overcleaning' this datafile. The first two are just a basic search.
You can search a line or the entire file.

In [13]:
def vsearch(line, term): # search for any term in a line/dataset
    search_term = re.compile(r'.*(%s).*' % (term), re.IGNORECASE )
    fields = filter(search_term.match, alldata[line])
    return(list(fields))

def searchall(term): # search entire CSV file. The output will tell you what line it's on.
    searchresults = []  
    for line in range(0,len(alldata)):
        if not vsearch(line, term) == []:
            searchresults = searchresults + [vsearch(line, term), 'line:' + str(line -1)]
    return(searchresults)


In [14]:
vsearch(1, 'state')

['This dataset provide a count of Veteran by their religious affiliation and state of residence. The dataset set covers all 50 states, District of Columbia and other territories.',
 'Veteran Religious Affiliation by State',
 'https://www.data.va.gov/sites/default/files/NCVAS%20Religion%20by%20State.xlsx',
 '<p>This dataset provide a count of Veteran by their religious affiliation and state of residence. The dataset set covers all 50 states, District of Columbia and other territories.</p>\n',
 'NCVAS Religion By State',
 'https://www.data.va.gov/sites/default/files/NCVAS%20Religion%20by%20State.xlsx']

What if we wanted all the 'by state' datasets?

In [26]:
searchall('by state')

[['Veteran Religious Affiliation by State', 'NCVAS Religion By State'],
 'line:0',
 ['Rural Veterans by State (2015)',
  'Rural Veterans by State (2015)',
  'Rural Veterans by State (2015)'],
 'line:1',
 ['2007 Veteran life insurance expenditures by state and county.  Expenditures were derived from Insurance Actuarial reports that used the total Insurance program cash disbursed.  The total cash disbursed was prorated by in-force data by state, which was further prorated by county and congressional district.',
  '2007 Veterans&#039; Insurance Expenditure by State and Congressional District'],
 'line:29',
 ['2007 Veteran life insurance expenditures by state and county.  Expenditures were derived from Insurance Actuarial reports that used the total Insurance program cash disbursed.  The total cash disbursed was prorated by in-force data by state, which was further prorated by county and congressional district.',
  '2007 Veterans&#039; Insurance Expenditure by State and County'],
 'line:30

Note the line is given, so I can pull up that dataset.  468 looks interesting.  This is a little hard to look at though.  A few more tools...

In [27]:
def keywords(line): # What keywords descibe this file?
    return(alldata[line][7])

def whatformat(line):# This will tell you what format a datafile is in
    return(alldata[line][31])


def getheaders(line): # get headers of csv or excel file
    earl = whaturl(line) # assuming the headers are on the first line
    print('Getting file...') 
    # for excel files
    if whatformat(line) in ('xlsx', 'xls'): 
        print('Excel file. Getting headers...')
        try:
            df = pd.read_excel(earl)
            return(df.columns.tolist())
        except:
            print('There was some problem reading this file.')
            print('Try accessing ' + earl + ' directly from your browser.')
    # for CSV files
    elif whatformat(line) == 'csv':
        print('CSV file. Getting headers...')
        try:
            res = requests.get(whaturl(line))
            df = pd.read_csv(io.StringIO(res.text))
            return(df.columns.tolist())
        except:
            print('There was some problem reading this file.')
            print('Try accessing ' + earl + ' directly from your browser.')
    else:
        print('Oh dear! This does not appear to be an excel or CSV file.')
        
def whaturl(line): # Where can I find the file for download
    return(alldata[line][-1])

In [28]:
keywords(468)

'benefits; compensation; county; pension; state; va; veteran'

In [29]:
whatformat(468)

'xls'

In [30]:
whaturl(468)

'https://www.va.gov/VETDATA/docs/Datagov/FY_2010_CNP_by_County.xlsx'

This function will pull the headers down into a pandas dataframe.  This one could use some help. It pulls  down the entire file first to give you the headers.  There are a number of reasons why this might not always work, including the assumption that the headers are always on the first row. 

In [31]:
getheaders(468)

Getting file...
Excel file. Getting headers...


['* denotes to fewer than 10 Veterans\nTotals include unknown gender and/or missing age',
 'Unnamed: 1',
 'Unnamed: 2',
 'Unnamed: 3',
 'Unnamed: 4',
 'Unnamed: 5',
 'Unnamed: 6',
 'Unnamed: 7',
 'Unnamed: 8',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13']

Yes, like that.  How about:

In [32]:
getheaders(6)

Getting file...
Excel file. Getting headers...


['Date', 'Gender', 'POS', 'Veterans']

Better.  Want to pull the entire dataframe?

In [33]:
def VAPandas(line): # Pull down CSV or Excel files into Pandas Dataframe
    earl = whaturl(line) # assuming the headers are on the first line
    print('Getting file...') 
    # for excel files
    if whatformat(line) in ('xlsx', 'xls'): 
        print('Excel file. Retrieving into Pandas Dataframe...')
        try:
            df = pd.read_excel(earl)
            return(df)
        except:
            print('There was some problem reading this file.')
            print('Try accessing ' + earl + ' directly from your browser.')
    # for CSV files
    elif whatformat(line) == 'csv':
        print('CSV file. Retrieving into Pandas Dataframe...')
        try:
            res = requests.get(whaturl(line))
            df = pd.read_csv(io.StringIO(res.text))
            return(df)
        except:
            print('There was some problem reading this file.')
            print('Try accessing ' + earl + ' directly from your browser.')
    else:
        print('Oh dear! This does not appear to be an excel or CSV file.')
          

In [34]:
VAPandas(6), type(6)

Getting file...
Excel file. Retrieving into Pandas Dataframe...


(           Date Gender                                                POS  \
 0    2015-09-30      F                          (a)\nAll Veterans\n (b+c)   
 1    2015-09-30      F    (b)\nWartime Veterans\n (i+j+k+m+n+p+q+r+t+u+v)   
 2    2015-09-30      F             (c )\nPeacetime Veterans\n (h+l+o+s+w)   
 3    2015-09-30      F                               (d)\n WWII\n (i+j+k)   
 4    2015-09-30      F                   (e)\nKorean Conflict \n(j+k+m+n)   
 5    2015-09-30      F                      (f)\nVietnam Era\n(k+n+p+q+r)   
 6    2015-09-30      F                     (g)\nGulf War Era\n(q+r+t+u+v)   
 7    2015-09-30      F                                      (h)\nPre-WWII   
 8    2015-09-30      F                                   (i)\nWWII\n only   
 9    2015-09-30      F                              (j)\nWWII &\n KC only   
 10   2015-09-30      F                                (k)\nWWII, KC,\nVNE   
 11   2015-09-30      F                          (l)\n Between\n

Also to be improved is the following function, which is supposed to tell you what, if any state or territory data is included in a particular header.  

In [35]:
def whatstate(line):
    which_states = []
    for x in alldata[line]:
        for state in states_territories:
            if state.lower() in x.lower():
                which_states = which_states + [state]
    if len(which_states) == 0:
        return("No state names found")
    else:
        return(list(set(which_states))) #turned into a set to remove dups, then back into list

So, for example. What states is covered in the first, third and seventh dataset? Note that we do include terrotories, and will have ways for sorting out territories and states. 

In [36]:
whatstate(564), whatstate(565), whatstate(1)

(['Delaware'], ['Florida'], 'No state names found')

So, what files have state data?

In [37]:
statedata = [] 
for i in range(0,len(alldata)):
    if not whatstate(i) == 'No state names found':
        statedata = statedata + [[whatstate(i)[0], i]]

In [38]:
sorted(statedata)

[['Alabama', 179],
 ['Alabama', 557],
 ['Alabama', 2098],
 ['Alabama', 2102],
 ['Alaska', 180],
 ['Alaska', 558],
 ['Alaska', 1701],
 ['Alaska', 2103],
 ['Alaska', 2104],
 ['Arizona', 210],
 ['Arizona', 559],
 ['Arizona', 2100],
 ['Arizona', 2105],
 ['California', 238],
 ['California', 561],
 ['California', 1713],
 ['California', 2107],
 ['California', 2782],
 ['California', 2783],
 ['California', 2784],
 ['Colorado', 252],
 ['Colorado', 321],
 ['Colorado', 322],
 ['Colorado', 447],
 ['Colorado', 562],
 ['Colorado', 883],
 ['Colorado', 1714],
 ['Colorado', 2108],
 ['Connecticut', 331],
 ['Connecticut', 563],
 ['Connecticut', 625],
 ['Connecticut', 771],
 ['Connecticut', 918],
 ['Connecticut', 1066],
 ['Connecticut', 1240],
 ['Connecticut', 1386],
 ['Connecticut', 1532],
 ['Connecticut', 1715],
 ['Connecticut', 2109],
 ['Connecticut', 2481],
 ['Connecticut', 2634],
 ['Delaware', 345],
 ['Delaware', 564],
 ['Delaware', 1716],
 ['Delaware', 2110],
 ['Florida', 446],
 ['Florida', 565],
 ['

To make searching for State datasets  easier, I've used a dictionary of state and territory abbreviations.

In [15]:
def StateSearch(StateAbbreviation):
    searchresults = []
    term = StateDict[StateAbbreviation]
    for line in range(0,len(alldata)):
        if not vsearch(line, term) == []:
            searchresults = searchresults + [vsearch(line, term), 'line:' + str(line -1)]
    return(searchresults)


In [16]:
StateSearch("MA")

[['Gravesite locations of Veterans and beneficiaries in MASSACHUSETTS, as of July 2017.',
  'Gravesite locations of Veterans and beneficiaries in MASSACHUSETTS, as of July 2017.',
  'https://www.data.va.gov/sites/default/files/ngl_massachusetts.csv',
  'NGL_Massachusetts_201708',
  'https://www.data.va.gov/sites/default/files/ngl_massachusetts.csv'],
 'line:578',
 ['This is a summary of the programs and services provided by VA in Massachusetts in fiscal year 2014.',
  'massachusetts; veterans',
  'State Summary:  Massachusetts',
  'State Summary:  Massachusetts',
  'https://www.va.gov/vetdata/docs/SpecialReports/State_Summaries_Massachusetts.pdf',
  'https://www.va.gov/vetdata/docs/SpecialReports/State_Summaries_Massachusetts.pdf'],
 'line:1729',
 ['Massachusetts-FY12 Benefits Summary',
  'https://www.data.va.gov/sites/default/files/Massachusetts-FY12_Benefits_summary.xlsx',
  'Massachusetts-FY12 Benefits Summary',
  'https://www.data.va.gov/sites/default/files/Massachusetts-FY12_Benef

# Other projects and future uses:
* Find all files with the same or similar headers
* Find data per county?
* Make the states/territories dataset into a dictionary format using state abbreviations
* Brent's CSV file is great, but maybe we could pull from the JSON directly?
* Fix whatever newbie coding mistakes I have made